In [2]:
import pandas as pd
import collections

df = pd.read_csv("../data/experimental/validation.tsv", sep="\t")

df = df[df["Number_of_Protein_Unique_Peptides"] >= 2]
df = df[df["Abundance_Ratio_log2"] > 2.3]
df = df[df["Abundance_Ratio_log2_median_corr"] > 1]
df = df[df["Abundance_Ratio_PValue"] < 0.05]
df = df[df["Abundance_Ratio_Adj_PValue"] < 0.25]
df = df[df["Rank_relative"] < 500]

slcs = []
with open("../data/examples/slc_cemm_interest.txt", "r") as f:
    for l in f:
        slcs += [l.rstrip()]

fid2smi = {}
for r in pd.read_csv("../data/experimental/validation_fid2smi.tsv", sep=",").values:
    fid2smi[r[0]] = r[1]

fid2pid = collections.defaultdict(list)
for r in df[["FragID", "Accession"]].values:
    fid2pid[r[0]] += [r[1]]

In [3]:
R = []
for k,v in fid2pid.items():
    v = set(v)
    s = v.intersection(slcs)
    r = (k, fid2smi[k], len(v), len(s), "; ".join(sorted(s)))
    R += [r]

dr = pd.DataFrame(R, columns=["fid", "smiles", "num_prot", "num_slc", "slc"])
dr.to_csv("../data/experimental/validation_hits_with_slc.tsv", sep="\t", index=False)